# Bag of Words Meets Bags of Popcorn

* 캐글 경진대회 링크 : https://www.kaggle.com/c/word2vec-nlp-tutorial

## 튜토리얼 파트 5 번외
* 캐글에 나와있는 튜토리얼 외에 점수를 좀 더 올려볼 수 있는 방법을 사용해 본다.
* tf-idf를 통해 벡터화 해보고 xgboost를 사용해 본다.

In [ ]:
import pandas as pd
"""
header = 0 은 파일의 첫 번째 줄에 열 이름이 있음을 나타내며 
delimiter = \t 는 필드가 탭으로 구분되는 것을 의미한다.
quoting = 3은 쌍따옴표를 무시하도록 한다.
"""
# QUOTE_MINIMAL (0), QUOTE_ALL (1), 
# QUOTE_NONNUMERIC (2) or QUOTE_NONE (3).

# 레이블인 sentiment 가 있는 학습 데이터
train = pd.read_csv('data/labeledTrainData.tsv', 
                    header=0, delimiter='\t', quoting=3)
# 레이블이 없는 테스트 데이터
test = pd.read_csv('data/testData.tsv', 
                   header=0, delimiter='\t', quoting=3)
print(train.shape)
print(test.shape)

(25000, 3)
(25000, 2)


In [ ]:
# 긍정과 부정 리뷰가 어떻게 들어있는지 카운트 해본다.
# 긍정과 부정리뷰가 각각 동일하게 12,500개씩 들어있다.
train['sentiment'].value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [ ]:
# 학습데이터에는 sentiment 데이터가 있다.
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


In [ ]:
# 테스트 데이터에는 sentiment가 없으며 이 sentiment를 예측하는 게 이 경진대회의 미션이다.
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
id        25000 non-null object
review    25000 non-null object
dtypes: object(2)
memory usage: 390.7+ KB


In [ ]:
# 데이터 전처리는 튜토리얼 파트1~4까지 공통으로 사용되기 때문에 별도의 파이썬 파일로 분리했다.
# 그리고 캐글에 있는 코드를 병렬처리하도록 멀티프로세싱 코드를 추가했다.
# 하지만 여기에서는 멀티프로세싱 코드만 불러와 사용하고 전처리는 태그만 제거해 주도록 했다.
from KaggleWord2VecUtility import KaggleWord2VecUtility
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def review_to_words( raw_review ):
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    review_text = wordnet_lemmatizer.lemmatize(review_text)
    return review_text

In [ ]:
# 학습데이터를 전처리 한다.
%time train['review_clean'] = KaggleWord2VecUtility.apply_by_multiprocessing(\
    train['review'], review_to_words, workers=4)

CPU times: user 98.1 ms, sys: 122 ms, total: 220 ms
Wall time: 7.1 s


In [ ]:
# 학습데이터와 동일하게 테스트 데이터에 대해서도 전처리 한다.
%time test['review_clean'] = KaggleWord2VecUtility.apply_by_multiprocessing(\
    test['review'], review_to_words, workers=4)

CPU times: user 108 ms, sys: 141 ms, total: 249 ms
Wall time: 6.79 s


In [ ]:
# 전처리한 학습 데이터 10개만을 불러와서 본다.
train['review_clean'][:10]

0    "With all this stuff going down at the moment ...
1    "\"The Classic War of the Worlds\" by Timothy ...
2    "The film starts with a manager (Nicholas Bell...
3    "It must be assumed that those who praised thi...
4    "Superbly trashy and wondrously unpretentious ...
5    "I dont know why people think this is such a b...
6    "This movie could have been very good, but com...
7    "I watched this video at a friend's house. I'm...
8    "A friend of mine bought this film for £1, and...
9    "This movie is full of references. Like \"Mad ...
Name: review_clean, dtype: object

In [ ]:
# 전처리한 테스트 데이터 10개만을 불러와서 본다.
test['review_clean'][:10]

0    "Naturally in a film who's main themes are of ...
1    "This movie is a disaster within a disaster fi...
2    "All in all, this is a movie for kids. We saw ...
3    "Afraid of the Dark left me with the impressio...
4    "A very accurate depiction of small time mob l...
5    "...as valuable as King Tut's tomb! (OK, maybe...
6    "This has to be one of the biggest misfires ev...
7    "This is one of those movies I watched, and wo...
8    "The worst movie i've seen in years (and i've ...
9    "Five medical students (Kevin Bacon, David Lab...
Name: review_clean, dtype: object

In [ ]:
# X_train과 X_test에 리뷰 데이터를 담아주고 이 데이터를 TF-IDF를 통해 임베딩(벡터화)해본다. 
X_train = train['review_clean']
X_test = test['review_clean']

## TF-IDF
TF(단어 빈도, term frequency)는 특정한 단어가 문서 내에 얼마나 자주 등장하는지를 나타내는 값으로, 이 값이 높을수록 문서에서 중요하다고 생각할 수 있다. 하지만 단어 자체가 문서군 내에서 자주 사용되는 경우, 이것은 그 단어가 흔하게 등장한다는 것을 의미한다. 이것을 DF(문서 빈도, document frequency)라고 하며, 이 값의 역수를 IDF(역문서 빈도, inverse document frequency)라고 한다. TF-IDF는 TF와 IDF를 곱한 값이다.

IDF 값은 문서군의 성격에 따라 결정된다. 예를 들어 '원자'라는 낱말은 일반적인 문서들 사이에서는 잘 나오지 않기 때문에 IDF 값이 높아지고 문서의 핵심어가 될 수 있지만, 원자에 대한 문서를 모아놓은 문서군의 경우 이 낱말은 상투어가 되어 각 문서들을 세분화하여 구분할 수 있는 다른 낱말들이 높은 가중치를 얻게 된다.

역문서 빈도(IDF)는 한 단어가 문서 집합 전체에서 얼마나 공통적으로 나타나는지를 나타내는 값이다. 전체 문서의 수를 해당 단어를 포함한 문서의 수로 나눈 뒤 로그를 취하여 얻을 수 있다.

* 출처 : [TF-IDF - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/TF-IDF)

\begin{equation*}
\text{tfidf}(w, d) = \text{tf} \times (\log\big(\frac{N + 1}{N_w + 1}\big) + 1)
\end{equation*}


* 싸이킷런 공식문서 : [4.2. Feature extraction — scikit-learn 0.19.1 documentation](http://scikit-learn.org/stable/modules/feature_extraction.html)

In [ ]:
from sklearn.pipeline import Pipeline

### TfidfVectorizer()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

char_vectorizer = Pipeline([('tfidf_char', TfidfVectorizer(
    analyzer='char', max_features=10000, ngram_range=(1, 9))) ])
char_vectorizer

Pipeline(memory=None,
     steps=[('tfidf_char', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 9), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None))])

In [ ]:
char_vectorizer.fit(X_train)

Pipeline(memory=None,
     steps=[('tfidf_char', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 9), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None))])

In [ ]:
%time X_train_char = char_vectorizer.transform(X_train)
X_train_char

CPU times: user 4min 7s, sys: 3.04 s, total: 4min 10s
Wall time: 4min 11s


<25000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 52373311 stored elements in Compressed Sparse Row format>

In [ ]:
%time X_test_char = char_vectorizer.transform(X_test)
X_test_char

CPU times: user 4min 14s, sys: 3.78 s, total: 4min 18s
Wall time: 4min 22s


<25000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 51556696 stored elements in Compressed Sparse Row format>

In [ ]:
word_vectorizer = Pipeline([('tfidf_char', TfidfVectorizer(
    analyzer='word', 
    max_features=30000, 
    ngram_range=(1, 2)))])
word_vectorizer

Pipeline(memory=None,
     steps=[('tfidf_char', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=30000, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None))])

In [ ]:
%time word_vectorizer.fit(X_train)

CPU times: user 19.2 s, sys: 334 ms, total: 19.5 s
Wall time: 19.5 s


Pipeline(memory=None,
     steps=[('tfidf_char', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=30000, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None))])

In [ ]:
%time X_train_word = word_vectorizer.transform(X_train)
X_train_word

CPU times: user 10.9 s, sys: 90.8 ms, total: 11 s
Wall time: 11 s


<25000x30000 sparse matrix of type '<class 'numpy.float64'>'
	with 5608076 stored elements in Compressed Sparse Row format>

In [ ]:
%time X_test_word = word_vectorizer.transform(X_test)
X_test_word

CPU times: user 10.7 s, sys: 110 ms, total: 10.8 s
Wall time: 10.8 s


<25000x30000 sparse matrix of type '<class 'numpy.float64'>'
	with 5457604 stored elements in Compressed Sparse Row format>

In [ ]:
from scipy.sparse import hstack

X_train = hstack([X_train_char, X_train_word])
X_train

<25000x40000 sparse matrix of type '<class 'numpy.float64'>'
	with 57981387 stored elements in COOrdinate format>

In [ ]:
X_test = hstack([X_test_char, X_test_word])
X_test

<25000x40000 sparse matrix of type '<class 'numpy.float64'>'
	with 57014300 stored elements in COOrdinate format>

In [ ]:
y_train = train["sentiment"]

print(y_train.shape)
y_train.head()

(25000,)


0    1
1    1
2    0
3    0
4    1
Name: sentiment, dtype: int64

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2018)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)

In [ ]:
%time forest = forest.fit(X_train.toarray(), train['sentiment'])

CPU times: user 6min 42s, sys: 16 s, total: 6min 58s
Wall time: 2min 37s


In [ ]:
%time result = forest.predict(X_test.toarray())

CPU times: user 7.37 s, sys: 8.68 s, total: 16 s
Wall time: 16.5 s


In [ ]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':result})
output.head()

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",1
3,"""7186_2""",0
4,"""12128_7""",1


In [ ]:
output_sentiment = output['sentiment'].value_counts()
print(output_sentiment[0] - output_sentiment[1])
output_sentiment

188


0    12594
1    12406
Name: sentiment, dtype: int64

In [ ]:
output.to_csv('data/tutorial_5_tfidf_rf_char_word.csv', index=False, quoting=3)

* 공식문서 : [XGBoost Documents](https://xgboost.readthedocs.io/en/latest/)
* [A Gentle Introduction to XGBoost for Applied Machine Learning - Machine Learning Mastery](https://machinelearningmastery.com/gentle-introduction-xgboost-applied-machine-learning/)

* https://speakerdeck.com/datasciencela/tianqi-chen-xgboost-overview-and-latest-news-la-meetup-talk

* datacamp의 XGboost 온라인 강의 : [Extreme Gradient Boosting with XGBoost](https://www.datacamp.com/courses/extreme-gradient-boosting-with-xgboost)

In [ ]:
import xgboost as xgb

params = {
    'booster': 'gblinear',
    'objective': 'multi:softmax',
    'eval_metric': 'merror',
    'lambda': 2.0,
    'alpha': 1.0,
    'lambda_bias': 6.0,
    'num_class': 5,
    'nthread': 8,
    'n_jobs': -1,
    'silent': 1,
}
xgb.XGBClassifier(params)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth={'booster': 'gblinear', 'objective': 'multi:softmax', 'eval_metric': 'merror', 'lambda': 2.0, 'alpha': 1.0, 'lambda_bias': 6.0, 'num_class': 5, 'nthread': 8, 'n_jobs': -1, 'silent': 1},
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)

%time booster = xgb.train(params, dtrain, num_boost_round=90)

CPU times: user 6min 27s, sys: 1.56 s, total: 6min 28s
Wall time: 2min 58s


In [ ]:
dtest = xgb.DMatrix(X_test.toarray())

predictions = booster.predict(dtest)

print(predictions.shape)
predictions[0:10]

(25000,)


array([1., 0., 1., 0., 1., 1., 0., 1., 0., 0.], dtype=float32)

In [ ]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':result})
output.head()

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",1
3,"""7186_2""",0
4,"""12128_7""",1


In [ ]:
output_sentiment = output['sentiment'].value_counts()
print(output_sentiment[0] - output_sentiment[1])
output_sentiment

188


0    12594
1    12406
Name: sentiment, dtype: int64

In [ ]:
output.to_csv("data/tutorial_5_tfidf_xgboost_char_word.csv", index=False, quoting=3)

In [ ]:
# 캐글 스코어 0.87564
270/578

0.4671280276816609